In [ ]:
import requests
import io
import base64
from PIL import Image

def face_swap(source_image_path, target_image_path, output_path):
    url = "http://localhost:7860/sdapi/v1/img2img"

    # 이미지 로드 및 base64 인코딩
    with open(source_image_path, "rb") as source_file:
        source_image = base64.b64encode(source_file.read()).decode('utf-8')
    
    with Image.open(target_image_path) as target_img:
        target_width, target_height = target_img.size
        target_image = base64.b64encode(target_img.tobytes()).decode('utf-8')

    payload = {
        "init_images": [target_image],
        "resize_mode": 0,
        "denoising_strength": 0.75,
        "mask": None,
        "mask_blur": 4,
        "inpainting_fill": 0,
        "inpaint_full_res": True,
        "inpaint_full_res_padding": 0,
        "inpainting_mask_invert": 0,
        "prompt": "",
        "seed": -1,
        "subseed": -1,
        "subseed_strength": 0,
        "batch_size": 1,
        "n_iter": 1,
        "steps": 20,
        "cfg_scale": 7,
        "width": target_width,
        "height": target_height,
        "restore_faces": False,
        "sampler_index": "Euler a",
        "alwayson_scripts": {
            "reactor": {
                "args": [
                    True,  # Enable ReActor
                    source_image,  # Source face image
                    "0",  # Comma separated face number(s) from swap-source image
                    "0",  # Comma separated face number(s) for target image
                    "0.5",  # Model background preservation (0 = keep source image, 1 = keep target image)
                    "0.5",  # Mask Blur
                    "0.5",  # Denoising Strength
                    "4",  # Mask Erode/Dilate
                    "0.5",  # Mask binary threshold
                    False,  # Match histogram
                    False,  # Sharpen
                    "CodeFormer",  # Face Restoration Model
                    True,  # Restore First
                    1.0,  # Restore visibility value
                    1.0,  # Codeformer Weight (0 = maximum effect, 1 = minimum effect)
                    False,  # Upscaler (disabled)
                    "1",  # Upscaler scale factor
                    0.5,  # Upscaler visibility
                    0   # Console Log Level (0 = default)
                ]
            }
        }
    }

    response = requests.post(url=url, json=payload)
    response.raise_for_status()
    
    # 결과 이미지 저장
    result = response.json()
    image = Image.open(io.BytesIO(base64.b64decode(result['images'][0])))
    image.save(output_path)
    print(f"Face swapped image saved to {output_path}")

# 사용 예시
source_image_path = "source_face.jpg"  # 소스 얼굴 이미지 경로
target_image_path = "target_image.jpg"  # 대상 이미지 경로
output_path = "face_swapped_result.png"  # 결과 이미지 저장 경로

face_swap(source_image_path, target_image_path, output_path)